<a href="https://colab.research.google.com/github/shazzad-hasan/github_demo/blob/main/en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/Questions

/content/drive/My Drive/Colab Notebooks/Questions


In [5]:
!ls

corpus.train.bn  corpus.train.en


In [6]:
file = open('corpus.train.en', encoding = 'utf8').read()

In [7]:
# Import required libraries
from operator import itemgetter
import nltk
import pandas as pd
from nltk.corpus import stopwords
from matplotlib import pyplot as plt

In [8]:
import nltk, re, pprint, string
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize
string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
#preprocess data
file_nl_removed = ""
for line in file:
  line_nl_removed = line.replace("\n", " ")      #removes newlines
  file_nl_removed += line_nl_removed
file_p = "".join([char for char in file_nl_removed if char not in string.punctuation])   #removes all special characters

In [10]:
sents = nltk.sent_tokenize(file_p)
print("The number of sentences is", len(sents)) 
#prints the number of sentences
words = nltk.word_tokenize(file_p)
print("The number of tokens is", len(words)) 
#prints the number of tokens
average_tokens = round(len(words)/len(sents))
print("The average number of tokens per sentence is",
average_tokens) 
#prints the average number of tokens per sentence
unique_tokens = set(words)
print("The number of unique tokens are", len(unique_tokens)) 
#prints the number of unique tokens

The number of sentences is 1722902
The number of tokens is 32637428
The average number of tokens per sentence is 19
The number of unique tokens are 508469


In [10]:
from nltk.util import ngrams
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
bigram=[]

tokenized_text = []
for sentence in sents:
    sentence = sentence.lower()
    sequence = word_tokenize(sentence) 
    tokenized_text.append(sequence) 
    bigram.extend(list(ngrams(sequence, 2)))  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def removal(x):     
#removes ngrams containing only stopwords
    y = []
    for pair in x:
        count = 0
        for word in pair:
            if word in stop_words:
                count = count or 0
            else:
                count = count or 1
        if (count==1):
            y.append(pair)
    return(y)
bigram = removal(bigram)

freq_bi = nltk.FreqDist(bigram)

print("Most common n-grams without stopword removal and without add-1 smoothing: \n")
print ("Most common bigrams: ", freq_bi.most_common(5))

In [ ]:
# %% Calculate the frequency of the words inside
frequency = {}
for word in sents:
    count = frequency.get(word , 0)
    frequency[ word ] = count + 1

In [ ]:
rank = 1
column_header = ['Rank', 'Frequency', 'Frequency * Rank']
df = pd.DataFrame( columns = column_header )
collection = sorted(frequency.items(), key=itemgetter(1), reverse = True)

In [ ]:
for word , freq in collection[:10]:
    df.loc[word] = [rank, freq, rank*freq]
    rank = rank + 1
    
print (df)

In [ ]:
# %% Python visualization with pyplot
plt.figure(figsize=(20,20))  #to increase the plot resolution
plt.ylabel("Frequency")
plt.xlabel("Words")
plt.xticks(rotation=90)    #to rotate x-axis values

for word , freq in collection[:30]:
    plt.bar(word, freq)    
plt.show()

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

#Stemming the words
#steming_words = [stemmer.stem(word) for word in words if not word in set(stopwords.words('english'))]

for word in words:
    print(word+' -> '+ stemmer.stem(word))

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

#Lemmatizing the words
lematizing_words = [lemmatizer.lemmatize(word) for word in words if not word in set(stopwords.words('english'))]

#for word in words:
#    print(word+' -> '+ lemmatizer.lemmatize(word))

In [ ]:
duplicates = []
cleaned = []
for s in sents:
    if s in cleaned:
        if s in duplicates:
            continue
        else:
            duplicates.append(s)
            print(duplicates)
    else:
        cleaned.append(s)

In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()

tf_idf =  vec.fit_transform(sents)
print(pd.DataFrame(tf_idf.toarray(), columns=vec.get_feature_names()))

In [23]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()

tf_idf =  vec.fit_transform(sents[:5])
print(pd.DataFrame(tf_idf.toarray(), columns=vec.get_feature_names()))

       also       and        as        be  better     bores  clemenceau  \
0  0.000000  0.000000  0.000000  0.000000     1.0  0.000000    0.000000   
1  0.000000  0.000000  0.258199  0.258199     0.0  0.000000    0.000000   
2  0.000000  0.226831  0.000000  0.000000     0.0  0.000000    0.000000   
3  0.000000  0.000000  0.000000  0.000000     0.0  0.000000    0.000000   
4  0.175006  0.141194  0.000000  0.000000     0.0  0.175006    0.350011   

   commotions   company  complained  ...      they        to     trump  \
0    0.000000  0.000000    0.000000  ...  0.000000  0.000000  0.000000   
1    0.000000  0.000000    0.000000  ...  0.000000  0.258199  0.000000   
2    0.000000  0.000000    0.000000  ...  0.281151  0.000000  0.281151   
3    0.000000  0.000000    0.000000  ...  0.000000  0.000000  0.000000   
4    0.175006  0.175006    0.175006  ...  0.000000  0.000000  0.000000   

       warm      will    wilson   wilsons    wishes      with       you  
0  0.000000  0.000000  0.00000

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
sents[1].lower()

'guaranteeing that every individual will be free to do as he wishes inevitably shortchanges equality.'